<a href="https://colab.research.google.com/github/LIMJIALIH/WIA1007/blob/main/Welcome_To_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Model Selection
Once your data is ready, you will need to choose an appropriate machine learning algorithm for your project. You may choose from supervised or unsupervised learning, and select the model based on the problem you are trying to solve.

Before we start with our machine learning, we need to make a copy of our dataset to avoid modifiying the original

In [1]:
final_df = scaled_df.copy()

NameError: name 'scaled_df' is not defined

In [ ]:
final_df.isna().sum()

In [ ]:
final_df.head()

In [ ]:
final_df.info()

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt


# Calculate the correlation matrix
correlation_matrix = final_df.select_dtypes(include=['number']).corr()

# Plot the correlation matrix as a heatmap
plt.figure(figsize=(12, 10))  # Adjust figure size as needed
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f")
plt.title('Correlation Matrix of Numerical Features')
plt.show()

###There are four categorical data location, property type, furnishing and size category.

### All we make them into True and False, and then drop all these categorical data.

In [ ]:
import pandas as pd

# Assuming your DataFrame is named 'scaled_df'

# Get a list of categorical features
categorical_features = ['Location', 'Property Type', 'Furnishing', 'Size Category']

# Perform one-hot encoding
encoded_df = pd.get_dummies(scaled_df, columns=categorical_features, drop_first=True, dtype=int)
# dtype=int ensures the encoded columns are integers (True/False represented as 1/0)
# drop_first=True avoids multicollinearity by dropping one level from each categorical feature

# Drop the original categorical columns
final_df = encoded_df.drop(categorical_features, axis=1, errors='ignore')
# errors='ignore' to handle cases where a column might have already been dropped

# Display the final DataFrame
print(final_df.head())

In [ ]:
# export the df_cleaned to 'balanced_cleaned_credit_card_data.csv'

final_df.to_csv('cleaned_data_for_ml.csv', index=False)

## Splitting Train and Test Data

Like every Machine Learning training steps, we will start with a train-test split on our data.

In [ ]:
from sklearn.model_selection import train_test_split
import pandas as pd

# Read the preprocessed and cleaned dataset
# The 'Price' column is not the first column, so no need to specify index_col
X = pd.read_csv('cleaned_data_for_ml.csv')

#Extract the target variable ('Price') and set it as variable 'y'
y = X.pop('Price')  # Now 'Price' is accessible as a regular column

#Split dataset into training and testing sets
#75% data used for training, 25% used for testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

We have split our dataset into training and testing sets, where the Train Data takes a random 75% of the dataset and the Test Data consists of the remaining 25% of the dataset. Our model will perform training on the Train Data and perform model evaluation on the Test Data to determine the better model.

## Determining Best Model

There is a lot of different classification models to test and see which model performs the best on our data. We have decided to use the R-squared and Root Mean Squared Error (RMSE) to measure our model performance.

In [ ]:
from sklearn.metrics import accuracy_score, mean_squared_error

Now we will train our data against multiple classification models to test which gives us the best performance.

Some common classification models are as below:

- Linear Regression
- Decision Tree Classifier
- Random Forest Classifier
- XGBRegressor

Below is the code to train our data against different classification models to help us find out which classification models perform best based on the 'R-squared' and 'Root-mean-square deviation(RMSE)' calculated.

In [ ]:
# Import all the necessary libraries for machine learning regression models
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor  # Assumes you have installed xgboost (pip install xgboost)
from sklearn.metrics import mean_squared_error, r2_score

# Create a list of regression models
regression_model_class = [
    LinearRegression,
    DecisionTreeRegressor,
    RandomForestRegressor,
    XGBRegressor,
]

# Model performance dictionary to store the results
model_performance = {
    'ModelName': ['LinearRegression',
                  'DecisionTreeRegressor',
                  'RandomForestRegressor',
                  'XGBRegressor'],
    'Model': [],
    'R-squared': [],  # Use R-squared for regression
    'RMSE': [],
}

# Loop through each regression model class
for ModelClass in regression_model_class:
    # Create model with specified arguments
    args = {'random_state': 161223} if ModelClass != LinearRegression else {}  # Handle XGBoost case
    model = ModelClass(**args)

    # Train data on model
    model.fit(X_train, y_train)

    # Model prediction using the trained model
    y_pred = model.predict(X_test)

    # Evaluation scoring
    model_performance['Model'].append(model)
    model_performance['R-squared'].append(r2_score(y_test, y_pred))  # Calculate R-squared
    model_performance['RMSE'].append(np.sqrt(mean_squared_error(y_test, y_pred)))

# Create DataFrame from the model performance dictionary
model_performance_df = pd.DataFrame(model_performance)
print(model_performance_df)

In [ ]:
import matplotlib.pyplot as plt

# Arrange the models from lowest to highest R-squared for better visualization.
model_performance_df.sort_values(by='R-squared', inplace=True)

# Create a figure and axis for the table
fig, ax = plt.subplots(figsize=(8, 6))

# Display the table in ascending order of R-squared
table_data = model_performance_df[['ModelName', 'R-squared', 'RMSE']]
table = ax.table(cellText=table_data.values, colLabels=table_data.columns, cellLoc='center', loc='center')
table.auto_set_font_size(False)
table.set_fontsize(10)

# Remove axes for better visualization
ax.axis('off')

plt.title('Model Performance Table')
plt.show()

Based on the table, we can see that Random Forest Regressor has the highest R-squared among all the model

We can visualize the model performance using bar charts. Model performs best if their:

- R-squared is highest
- RMSE value is lowest

The code provided below is determine which model performs the best based on the 'R-squared' value and RMSE value. After performing the code above, the DataFrame now contains the model names, the actual model instances, R-squared scores, and RMSE values for each model. This makes it easier to analyze and visualize the model performance into charts as provided in the code below.

In [ ]:
# Visualize model performa nce by creating two subplots ax1 and ax2
fig, (ax1, ax2) = plt.subplots(2, layout='constrained')

#Arrange the models from lowest to highest R-squared for better visualization.
model_performance_df.sort_values(by='R-squared', inplace=True)
#Horizontal bar chart is created on the first subplot (ax1). ( y-axis: 'ModelName', x-axis : 'R-squared')
ax1.barh(model_performance_df['ModelName'], model_performance_df['R-squared'], color=sns.color_palette('flare'))
#Set title of subplot ax1
ax1.set_title('R-squared')

#Arrange the models from highest to lowest RMSE for better visualization.
model_performance_df.sort_values(by='RMSE', ascending=False, inplace=True)
#Horizontal bar chart is created on the second subplot (ax2). ( y-axis: 'ModelName', x-axis : 'RMSE')
ax2.barh(model_performance_df['ModelName'], model_performance_df['RMSE'], color=sns.color_palette('flare'))
#Set title of subplot ax2
ax2.set_title('RMSE')

## Selected Model

Since the R-squared for RandomForestRegressor is the highest and the RMSE value is the lowest, RandomForestRegressor is the best performing model. Without further ado, We will choose RandomForestRegressor for our model.

## Model Training and Evaluation

Once the model is selected, it needs to be trained on the preprocessed data, and its performance should be evaluated using appropriate metrics. The model can be fine-tuned to improve its performance through Hyperparameter Tuning.

## Hyperparameter Tuning

In [ ]:
from sklearn.model_selection import GridSearchCV

params = {
    'max_depth': range(1, 10),
}

#Create an instance of GridSearchCV with RandomForestRegressor, searching over the specified parameter grid
#Using 4-fold cross-validation and R-squared as the scoring metric
clf = GridSearchCV(RandomForestRegressor(), param_grid=params, scoring='r2', cv=4)

# Fit the GridSearchCV instance on the training data to find the best hyperparameters
clf.fit(X_train, y_train)

# Display the best hyperparameters and corresponding best R-squared score
clf.best_params_, clf.best_score_

The first hyperparameter values found were {'max_depth': 9} with a corresponding best accuracy score of approximately 0.999986.

In [ ]:
params = {
    'max_depth': [9],
    'min_samples_split': range(2, 11),
    'min_samples_leaf': [i for i in range(1, 10)],
    'ccp_alpha': [i/10.0 for i in range(0, 10)]
}

#Create an instance of GridSearchCV with RandomForestRegressor, searching over the specified parameter grid
#Using 4-fold cross-validation and R-squared as the scoring metric
clf = GridSearchCV(RandomForestRegressor(), param_grid=params, scoring='r2', cv=4)
# Fit the GridSearchCV instance on the training data to find the best hyperparameters
clf.fit(X_train, y_train)

# Display the best hyperparameters and corresponding best R-squared score
clf.best_params_, clf.best_score_

We can extend the GridSearch to include additional hyperparameters: 'max_depth','min_samples_split', 'min_samples_leaf', and 'ccp_alpha'.

The best hyperparameter values found in this case were {'ccp_alpha': 0.0, 'max_depth': 9, 'min_samples_leaf': 1, 'min_samples_split': 7} with a corresponding best accuracy score of approximately 0.999991.

In [ ]:
model.score(X_test, y_test)

The best hyperparameters found during the tuning process are utilized for training the random forest model as shown above.

## Experimenting

Make a df copy and try a different normalisation technique to observe the change in accuracy

In [ ]:
df_normStandardScaler = scaled_df.copy()

In this case, we choose standard scaler as another method to normalise the data as our previous normalised data is using min max scaler

Next we need to change the columns to standardize

In [ ]:
from sklearn.preprocessing import StandardScaler

# Create a StandardScaler
scaler = StandardScaler()

# Create a copy of the DataFrame for standardization
df_normstandard = scaled_df.copy()

# Perform standardization on selected 3 columns
df_normstandard[['Distance from Home', 'Distance from Last Transaction', 'Ratio to Median Purchase Price']] = scaler.fit_transform(df_normstandard[['Distance from Home', 'Distance from Last Transaction', 'Ratio to Median Purchase Price']])

# Display the first 5 rows of the standardized DataFrame
df_normstandard.head()

In [ ]:
X_exp = df_normstandard
y_exp = X_exp.pop('Fraud')

In [ ]:
from sklearn.model_selection import train_test_split
X_train_exp, X_test_exp, y_train_exp, y_test_exp = train_test_split(X_exp, y_exp, test_size=0.2)

Need to know the ccp_alpha... to run next code first

In [ ]:
model_exp = sklearn.tree.DecisionTreeClassifier(ccp_alpha=0.0, max_depth=5, min_samples_leaf=1, min_samples_split=7)
model_exp.fit(X_train_exp, y_train_exp)

In [ ]:
model_exp.score(X_test_exp, y_test_exp)

Need to change also next code

In [ ]:
# a table to compare the accuracy value of model.score and model_exp.score

model_score = model.score(X_test, y_test)
model_exp_score = model_exp.score(X_test_exp, y_test_exp)

table = pd.DataFrame({'Model': ['Decision Tree (Min Max Scaler)', 'Decision Tree (Standard Scaler)'],
                     'Accuracy': [model_score, model_exp_score]})

table


Based on the experimental trial of using Standard Scaler as our scaling method, the accuracy seem to ...

## Pipelining

Since our data is preprocessed before entering into our ML model, we can create a pipeline. This allow us to generalize a dataset input from data preprocess to the ML model prediction and receive a output.

In [ ]:
from sklearn.pipeline import Pipeline

# Create Pipeline
pipeline = Pipeline(steps=[
    ('model', DecisionTreeClassifier(random_state=161223, **clf.best_params_)),
])

In [ ]:
pipeline

#Export Pipeline

Now we can export our pipeline for deployment using joblib library.

In [ ]:
import joblib
# Fit the machine learning pipeline to the training data
pipeline.fit(X_train, y_train)

# Save the trained pipeline to a file using joblib named 'pipeline.pkl' and compression is applied
joblib.dump(pipeline, 'pipeline.pkl', compress=True)

## Load Pipeline

We can load our pipeline and predict it with our new test data to see if it works as expected.

In [ ]:
# Load the pre-trained machine learning pipeline
pipeline_loaded = joblib.load('pipeline.pkl')

# Calculate the accuracy score of the model
pipeline_loaded.score(X_test, y_test)

As what we expect, our pipeline is able to preprocess the data correctly and give us a fairly accurate result.

# Final Deliverable
Finally, you will need to present your findings and results in a clear and concise manner. Document it using Google Colab/Kaggle/Jupyter Notebook. You only
submit the link to your notebook. Please make grant access for me to view and assess your work.

## Evaluation

The accuracy of the model is 0.9999930434782609. This shows that the accuracy of the trained model is impressively high, achieving a score of approximately 99.9993%.This high accuracy suggests that the model performs exceptionally well.